<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/paradetox-preprocess-maskedComments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!apt install git-lfs

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset
import re

dataset = load_dataset("HamdanXI/paradetox-preprocess")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/19744 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en_toxic_comment', 'en_neutral_comment'],
        num_rows: 19744
    })
})

In [24]:
import difflib

def get_diff_ops(original, edited):
    try:
        ops = []
        original_words = original.split()
        edited_words = edited.split()
        s = difflib.SequenceMatcher(None, original_words, edited_words)

        for tag, i1, i2, j1, j2 in s.get_opcodes():
            if tag == 'delete':
                for i in range(i1, i2):
                    ops.append(("delete", original_words[i], i))
            elif tag == 'insert':
                for j in range(j1, j2):
                    ops.append(("insert", edited_words[j], j1))
            elif tag == 'replace':
                max_replace = max(i2 - i1, j2 - j1)
                for i in range(max_replace):
                    orig_word = original_words[i1 + i] if i1 + i < i2 else None
                    new_word = edited_words[j1 + i] if j1 + i < j2 else None
                    if orig_word and new_word:
                        ops.append(("replace", orig_word, i1 + i, new_word))
                    elif orig_word:
                        ops.append(("delete", orig_word, i1 + i))
                    elif new_word:
                        ops.append(("insert", new_word, j1 + i))
        return ops
    except Exception as e:
        print(f"Error with original: {original} and edited: {edited}. Error: {e}")
        return []


def apply_diff_ops(example):
    original = example['en_toxic_comment']
    edited = example['en_neutral_comment']

    ops_list = get_diff_ops(original, edited)
    original_words = original.split()

    # Track the shifts caused by previous insertions
    shifts = 0
    # Initialize an empty list to hold formatted operations for the new column
    formatted_ops = []

    for op in ops_list:
      op_type, word, index, *rest = op
      if op_type == 'delete':
          original_words[index + shifts] = '[DELETE]'
          formatted_ops.append((op_type, word, str(index)))  # Convert index to string
      elif op_type == 'insert':
          original_words.insert(index + shifts, '[INSERT]')
          shifts += 1  # Increment the shift for subsequent indexes
          formatted_ops.append((op_type, word, str(index + shifts)))  # Convert index to string
      elif op_type == 'replace':
          original_words[index + shifts] = '[REPLACE]'
          formatted_ops.append((op_type, word, str(index + shifts), rest[0]))  # Convert index to string


    # Join the masked_comment list back into a single string
    masked_comment = ' '.join(original_words)

    # Add the masked comment and formatted operations to the example
    example['edit_ops'] = formatted_ops
    example['masked_comment'] = masked_comment
    return example

In [25]:
dataset_with_ops = dataset.map(apply_diff_ops)

Map:   0%|          | 0/19744 [00:00<?, ? examples/s]

In [27]:
dataset_with_ops.push_to_hub("paradetox-preprocess-maskedComments")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

In [26]:
dataset_with_ops['train'][0]

{'en_toxic_comment': 'he had steel balls too!',
 'en_neutral_comment': 'he was brave too!',
 'edit_ops': [['replace', 'had', '1', 'was'],
  ['replace', 'steel', '2', 'brave'],
  ['delete', 'balls', '3']],
 'masked_comment': 'he [REPLACE] [REPLACE] [DELETE] too!'}